In [34]:
import numpy as np
import os
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import MinMaxScaler, RobustScaler, scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import modules.feature_extraction as fe

#### Features aus csv lesen

In [35]:
df = pd.read_csv(r"data\features.csv", sep=';', header=None)
df = df.iloc[1:, 3:]
df = df.reindex(columns=[4, 5, 6, 7, 3])
df = df.apply(pd.to_numeric, errors='coerce')
df = df[df.iloc[:, -1] < 3]
df = df.sample(frac=1)

#### x y Werte zum Trainieren trennen

In [36]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1]

#### Train / Test Daten trennen

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

#### Support Vector Machine trainieren

In [38]:
model = SVC(verbose=1, C=1000, gamma=0.002)
model.fit(x_train, y_train)

[LibSVM]

SVC(C=1000, gamma=0.002, verbose=1)

#### Mit Testdaten vergleichen

In [39]:
y_pred = model.predict(x_test)

#### Accuracy-, Precision-, Recall- und F1-Score ausgeben

In [40]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:\t{}\nPrecision:\t{}\nRecall:\t\t{}\nF1-Score:\t{}".format(accuracy, precision, recall, f1))

Accuracy:	0.5112781954887218
Precision:	0.47464429027993366
Recall:		0.5112781954887218
F1-Score:	0.4650992035686272


#### Testdaten von Fr. Burghart hier

In [41]:
relImgPath = os.path.join("data", "images", "test")

imgPaths = []
for dirpath, dirnames, filenames in os.walk(relImgPath):
    if dirnames:
        classes = {}
        for index, name in enumerate(dirnames):
            classes[name]=index
    for filename in filenames:# [f for f in filenames if f.endswith(suportedImgFomats)]:
        imgPaths.append(os.path.join(dirpath, filename))

# TODO delete SPLITIT folder
print("Class names and indices:", classes)

features = ["Relative Image Path",
           "Class Name",
           "Class Index",
           "Aspect Ratio",
           "Number of Corners (Harris)",
           "Number of Corners (Shi-Tomasi)",
           "Perimeter Area Ratio"]

df = pd.DataFrame(columns=features)

for i, path in enumerate(imgPaths):
    c = path.split(os.sep)[-2]
    img = fe.prepared_image(path)
    ratio = fe.aspect_ratio(img)#path) 
    numCornersH = fe.num_corners(img, detector="harris")#fe.harris_corner_detection(img)#,path)#path)
    numCornersST = fe.num_corners(img, detector="shi-tomasi")#fe.shi_tomasi_corner_detection(img)#path)
    ratioPerim = fe.perimeter_area_ratio(img)
    
    row = pd.Series([path, c, classes[c], ratio, numCornersH, numCornersST, ratioPerim], index = features)
    df = df.append(row, ignore_index=True)
    
    #for imgName in ["original", "prepared", "canny", "canny closed gaps", 
    #                "contours", "max area contour", "harris", "shi-tomasi"]:
    #    fe.display_image(fe.images[imgName], title=imgName)
    
    print(str(format((100./len(imgPaths))*i, ".2f"))+" %", end="\r")


print("Information on the feature dataframe:")
df.info()

Class names and indices: {'bottleOpener': 0, 'canOpener': 1, 'corcScrew': 2, 'multiTool': 3}
Information on the feature dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Relative Image Path             19 non-null     object 
 1   Class Name                      19 non-null     object 
 2   Class Index                     19 non-null     object 
 3   Aspect Ratio                    19 non-null     float64
 4   Number of Corners (Harris)      19 non-null     object 
 5   Number of Corners (Shi-Tomasi)  19 non-null     object 
 6   Perimeter Area Ratio            19 non-null     float64
dtypes: float64(2), object(5)
memory usage: 1.2+ KB


In [42]:
#df = df.iloc[1:, 3:]
df = df.reindex(columns=["Aspect Ratio", "Number of Corners (Harris)",
           "Number of Corners (Shi-Tomasi)",
           "Perimeter Area Ratio", "Class Index"])
df = df.apply(pd.to_numeric, errors='coerce')
df = df[df.iloc[:, -1] < 3]
df = df.sample(frac=1)

In [45]:
x_test = df.iloc[:, :-1]
y_test = df.iloc[:, -1]

y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:\t{}\nPrecision:\t{}\nRecall:\t\t{}\nF1-Score:\t{}".format(accuracy, precision, recall, f1))

Accuracy:	0.7857142857142857
Precision:	0.8660714285714286
Recall:		0.7857142857142857
F1-Score:	0.787480376766091


In [48]:
class_names = dict((v, k) for k, v in classes.items())

for idx, pred in enumerate(y_pred):
    print (f"Bild Nr. {idx}: Tatsächlich: {class_names[y_test[idx]]} | Prediction: {class_names[pred]}")

Bild Nr. 0: Tatsächlich: bottleOpener | Prediction: bottleOpener
Bild Nr. 1: Tatsächlich: bottleOpener | Prediction: bottleOpener
Bild Nr. 2: Tatsächlich: bottleOpener | Prediction: bottleOpener
Bild Nr. 3: Tatsächlich: bottleOpener | Prediction: corcScrew
Bild Nr. 4: Tatsächlich: bottleOpener | Prediction: canOpener
Bild Nr. 5: Tatsächlich: canOpener | Prediction: corcScrew
Bild Nr. 6: Tatsächlich: canOpener | Prediction: bottleOpener
Bild Nr. 7: Tatsächlich: canOpener | Prediction: canOpener
Bild Nr. 8: Tatsächlich: canOpener | Prediction: canOpener
Bild Nr. 9: Tatsächlich: canOpener | Prediction: bottleOpener
Bild Nr. 10: Tatsächlich: corcScrew | Prediction: bottleOpener
Bild Nr. 11: Tatsächlich: corcScrew | Prediction: bottleOpener
Bild Nr. 12: Tatsächlich: corcScrew | Prediction: bottleOpener
Bild Nr. 13: Tatsächlich: corcScrew | Prediction: corcScrew
